In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
from io import StringIO
import pygwalker as pyg
import os
from tqdm import tqdm
import polars.selectors as cs
from scipy.stats import zscore 

reading competition editions json

In [2]:
ce_json_path = './skillcorner-competition-editions.json'
ce_df1 = pl.read_json(ce_json_path)
ce_df1.head(1)
#ce_df.columns

count,next,previous,results
i64,null,null,list[struct[4]]
938,null,null,"[{0,{0,""FIFA"",""World Cup"",""male"",""adult""},{10,2014,2014,""2014""},""FIFA - World Cup - 2014""}, {1,{1,""ENG"",""Premier League"",""male"",""adult""},{2,2015,2016,""2015/2016""},""ENG - Premier League - 2015/2016""}, … {1129,{221,""INT"",""Pinatar National Team Friendlies"",""male"",""U21""},{128,2025,2025,""2025""},""INT - Pinatar National Team Friendlies - 2025""}]"


In [3]:
#ce_df2 = ce_df1.explode('results')
ce_df2 = ce_df1.select('results').explode('results')
ce_df2 = ce_df2.unnest('results')
ce_df2.head(1)

id,competition,season,name
i64,struct[5],struct[4],str
0,"{0,""FIFA"",""World Cup"",""male"",""adult""}","{10,2014,2014,""2014""}","""FIFA - World Cup - 2014"""


In [4]:
ce_name_id = ce_df2.select([
    pl.col('name'),
    pl.col('id'),
    ]
)
ce_name_id.head(1)

name,id
str,i64
"""FIFA - World Cup - 2014""",0


In [5]:
ce_competition = ce_df2.select('competition').unnest('competition')
ce_competition = ce_competition.rename({col: f"competition_{col}" for col in ce_competition.columns})
ce_competition.head(1)

competition_id,competition_area,competition_name,competition_gender,competition_age_group
i64,str,str,str,str
0,"""FIFA""","""World Cup""","""male""","""adult"""


In [6]:
ce_season = ce_df2.select('season').unnest('season')
ce_season = ce_season.rename({col: f"season_{col}" for col in ce_season.columns})
ce_season.head(1)

season_id,season_start_year,season_end_year,season_name
i64,i64,i64,str
10,2014,2014,"""2014"""


In [7]:
ce = pl.concat([
    ce_name_id,
    ce_season,
    ce_competition
],
how = "horizontal",)
ce.head(10)

name,id,season_id,season_start_year,season_end_year,season_name,competition_id,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,i64,str,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,10,2014,2014,"""2014""",0,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2,2015,2016,"""2015/2016""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,12,2016,2016,"""2016""",2,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,3,2016,2017,"""2016/2017""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,4,2017,2018,"""2017/2018""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""FRA - Ligue 1 - 2016/2017""",5,3,2016,2017,"""2016/2017""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""FRA - Ligue 1 - 2017/2018""",6,4,2017,2018,"""2017/2018""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""ESP - LaLiga - 2016/2017""",7,3,2016,2017,"""2016/2017""",4,"""ESP""","""LaLiga""","""male""","""adult"""
"""ESP - LaLiga - 2017/2018""",8,4,2017,2018,"""2017/2018""",4,"""ESP""","""LaLiga""","""male""","""adult"""


reading json player file- test on 1 file


In [8]:
position_json_testpath = './skillcorner-20250214/metrics-ce249-rAM.json'

position_test_df = pl.read_json(position_json_testpath)

In [9]:
position_test_df.columns
#position_test_df.head(10)

['Minutes',
 'Count Sprint OTIP 2',
 'Count Medium Acceleration OTIP',
 'Distance',
 'Distance 1',
 'Count Sprint 1',
 'Sprinting Distance TIP 1',
 'player_birthdate',
 'match_name',
 'Count HSR 1',
 'Count HSR TIP',
 'Count High Deceleration TIP',
 'Count Medium Deceleration',
 'Sprinting Distance 1',
 'Count High Deceleration 1',
 'Minutes OTIP 2',
 'Count HSR TIP 2',
 'Count High Deceleration OTIP',
 'Sprinting Distance OTIP 1',
 'Sprinting Distance 2',
 'Count Medium Acceleration 1',
 'Distance TIP 1',
 'HSR Distance TIP',
 'Running Distance TIP 1',
 'Minutes 2',
 'Count Medium Deceleration TIP 1',
 'match_id',
 'Distance OTIP 1',
 'Count Medium Acceleration 2',
 'Count Medium Deceleration 1',
 'Count Medium Acceleration TIP',
 'Count High Acceleration OTIP',
 'Sprinting Distance TIP 2',
 'Count HSR OTIP',
 'Count High Acceleration OTIP 2',
 'HSR Distance OTIP 2',
 'Count Medium Acceleration OTIP 1',
 'competition_edition_id',
 'Count Sprint',
 'Count Medium Acceleration',
 'Sprint

In [10]:
#define the path to the entire directory
json_dir_path = './skillcorner-20250214/'

### Look at correlations between parameters
Looking at the correlations across all competitions across all positions to see if we can leave out certain parameters

In [11]:
json_list = os.listdir(json_dir_path)
#json_list

In [12]:
"""dfs = []
for jsonf in json_list:
    json_path = f"{json_dir_path}{jsonf}"
    dfs.append(pl.read_json(json_path))

corr_df = pl.concat(dfs, how="align_full")"""

'dfs = []\nfor jsonf in json_list:\n    json_path = f"{json_dir_path}{jsonf}"\n    dfs.append(pl.read_json(json_path))\n\ncorr_df = pl.concat(dfs, how="align_full")'

In [13]:
#loop through the json files and concat them into one big df

# creating an empty data frame to concat the json dataframes to
read_json_list = []

for jsonf in tqdm(json_list):
    json_path = f"{json_dir_path}{jsonf}"
    json_df = pl.read_json(json_path)

    read_json_list.append(json_df)

corr_df = pl.concat(read_json_list)


corr_df.shape
#corr_df.head(2)
"""
output_path = "json_output.csv"
corr_df.write_csv(output_path)"""

  4%|▍         | 66/1652 [00:02<01:03, 25.00it/s]

100%|██████████| 1652/1652 [00:45<00:00, 35.93it/s]


'\noutput_path = "json_output.csv"\ncorr_df.write_csv(output_path)'

In [14]:
pl.Config.set_tbl_cols(250)
corr_df.head()

Minutes,Count Sprint OTIP 2,Count Medium Acceleration OTIP,Distance,Distance 1,Count Sprint 1,Sprinting Distance TIP 1,player_birthdate,match_name,Count HSR 1,Count HSR TIP,Count High Deceleration TIP,Count Medium Deceleration,Sprinting Distance 1,Count High Deceleration 1,Minutes OTIP 2,Count HSR TIP 2,Count High Deceleration OTIP,Sprinting Distance OTIP 1,Sprinting Distance 2,Count Medium Acceleration 1,Distance TIP 1,HSR Distance TIP,Running Distance TIP 1,Minutes 2,Count Medium Deceleration TIP 1,match_id,Distance OTIP 1,Count Medium Acceleration 2,Count Medium Deceleration 1,Count Medium Acceleration TIP,Count High Acceleration OTIP,Sprinting Distance TIP 2,Count HSR OTIP,Count High Acceleration OTIP 2,HSR Distance OTIP 2,Count Medium Acceleration OTIP 1,competition_edition_id,Count Sprint,Count Medium Acceleration,Sprinting Distance OTIP,Count Medium Acceleration TIP 1,HSR Distance OTIP,Count High Acceleration TIP 1,Sprinting Distance OTIP 2,Count Medium Deceleration OTIP,Minutes OTIP,Count Medium Deceleration 2,Running Distance,Running Distance 2,HSR Distance 2,Count High Acceleration OTIP 1,Count Sprint TIP,Running Distance OTIP 1,team_name,Count Medium Deceleration OTIP 2,season_name,group,Distance OTIP 2,player_name,Distance TIP 2,Count Medium Deceleration TIP,Sprinting Distance,HSR Distance TIP 2,Count Medium Deceleration TIP 2,Running Distance OTIP,quality_check,Count Medium Deceleration OTIP 1,Count Sprint OTIP,Minutes OTIP 1,Count Sprint TIP 1,short_name,team,Count High Acceleration 1,Count HSR OTIP 1,Count HSR,Minutes TIP 1,Running Distance TIP,Count High Deceleration OTIP 2,player_id,Running Distance 1,Count Medium Acceleration OTIP 2,Count High Acceleration,PSV-99,Count High Acceleration 2,HSR Distance TIP 1,Count High Acceleration TIP,Count Sprint OTIP 1,Count High Acceleration TIP 2,Count Medium Acceleration TIP 2,Minutes TIP,position,HSR Distance 1,Distance 2,Minutes 1,Count Sprint 2,Minutes TIP 2,Count HSR OTIP 2,Count High Deceleration TIP 1,date,Count High Deceleration,Count High Deceleration TIP 2,Running Distance TIP 2,competition_id,Count Sprint TIP 2,HSR Distance,Count HSR 2,Count HSR TIP 1,season_id,Distance OTIP,Count High Deceleration OTIP 1,Distance TIP,Count High Deceleration 2,Running Distance OTIP 2,Sprinting Distance TIP,HSR Distance OTIP 1,team_id
f64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,str,i64,str,i64,i64,i64,i64,i64,i64,bool,i64,i64,f64,i64,str,str,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,f64,str,i64,i64,f64,i64,f64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
96.63,1,47,13026,6810,3,11,"""1995-02-17""","""KV Mechelen v Royal Antwerp FC""",33,19,3,89,92,8,17.3,6,4,27,56,52,1874,260,369,49.48,8,44999,2983,48,42,22,0,19,34,0,218,24,249,5,100,45,12,443,0,18,34,35.57,47,2415,1061,358,0,2,842,"""Royal Antwerp FC""",17,"""2021/2022""","""Midfield""",2744,"""Pieter Gerkens""",1558,17,148,121,9,1504,true,17,2,18.27,1,"""P. Gerkens""","""Royal Antwerp FC""",3,17,59,13.13,661,1,8701,1354,23,4,25.7,1,139,0,1,0,10,24.21,"""AM""",387,6216,47.15,2,11.08,17,2,"""2021-07-25""",11,1,292,16,1,745,26,13,8,5727,3,3432,3,662,30,225,462
96.19,0,32,9969,4842,3,17,"""1989-07-25""","""Oud-Heverlee Leuven v SV Zulte…",20,20,2,77,48,5,15.07,11,1,24,154,36,1660,265,267,50.07,9,45002,1795,41,31,25,1,119,17,1,63,15,249,8,77,24,11,167,1,0,25,29.59,46,1101,612,275,0,4,178,"""Oud-Heverlee Leuven""",14,"""2021/2022""","""Midfield""",1739,"""Xavier Mercier""",1775,20,202,184,11,402,true,11,1,14.52,1,"""X. Mercier""","""Oud-Heverlee Leuven""",3,10,39,13.2,610,0,10932,489,17,5,28.3,2,81,2,1,1,14,25.52,"""AM""",201,5127,46.12,5,12.32,7,1,"""2021-07-24""",8,1,343,16,3,476,19,9,8,3534,1,3435,3,224,136,104,27
71.07,1,23,8709,5620,4,26,"""1996-01-18""","""Oud-Heverlee Leuven v SV Zulte

In [15]:
nonull_numeric = corr_df.drop_nulls().select(cs.numeric())

In [16]:
correlations = nonull_numeric.corr()

In [17]:
# only retaining meaningfull correlation

# Initialize an empty list to store the filtered correlation pairs
filtered_data = []

# Iterate over the rows and columns to extract variable pairs and correlations
for i, row in enumerate(correlations.columns):
    for j, col in enumerate(correlations.columns):
        if i < j:  # Skip self-correlation and duplicate pairs (ensure we only process one pair)
            corr_value = correlations[row][j]  # Access the correlation value at row i, column j
            if corr_value > 0.8:  # Only keep correlations > 0.8
                filtered_data.append([row, col, corr_value])

# Create a Polars DataFrame with the filtered data
filtered_df = pl.DataFrame(filtered_data, schema=["Variable 1", "Variable 2", "Correlation"])

# Write the filtered data to a CSV file (since Polars doesn't support writing to Excel directly)
filtered_df.write_excel(workbook = "filtered_correlations.xlsx")

print("Filtered correlations saved to 'filtered_correlations.csv'.")

/tmp/ipykernel_7200/36976019.py:15: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  filtered_df = pl.DataFrame(filtered_data, schema=["Variable 1", "Variable 2", "Correlation"])


Filtered correlations saved to 'filtered_correlations.csv'.


In [18]:
nonull_numeric_np = nonull_numeric.to_numpy()

In [19]:
#get back the correlations
#get back the z-scores
z_scores = zscore(nonull_numeric_np)

In [20]:
# Find outliers
outliers = nonull_numeric_np[(np.abs(zscore(nonull_numeric_np)) >= 5).all(axis=1)]

# Check the shape
print(outliers.shape)

(0, 106)


In [21]:
pyg.walk(position_test_df)

Box(children=(HTML(value='\n<div id="ifr-pyg-000632e2df303b2feTGbDsYXlZyIF1p7" style="height: auto">\n    <hea…

In [22]:
ce.head()

name,id,season_id,season_start_year,season_end_year,season_name,competition_id,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,i64,str,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,10,2014,2014,"""2014""",0,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2,2015,2016,"""2015/2016""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,12,2016,2016,"""2016""",2,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,3,2016,2017,"""2016/2017""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,4,2017,2018,"""2017/2018""",1,"""ENG""","""Premier League""","""male""","""adult"""


In [23]:
# change column name of ce df id collumn for easy join
ce = ce.rename({"id":"competition_edition_id"})

In [24]:
ce.head()

name,competition_edition_id,season_id,season_start_year,season_end_year,season_name,competition_id,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,i64,str,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,10,2014,2014,"""2014""",0,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2,2015,2016,"""2015/2016""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,12,2016,2016,"""2016""",2,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,3,2016,2017,"""2016/2017""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,4,2017,2018,"""2017/2018""",1,"""ENG""","""Premier League""","""male""","""adult"""


In [25]:
#join corr_df (competition_edition_id) and ce_df (id)
work_df = corr_df.join(ce, on = "competition_edition_id")

In [26]:
pl.Config.set_tbl_cols(250)
work_df.head()

Minutes,Count Sprint OTIP 2,Count Medium Acceleration OTIP,Distance,Distance 1,Count Sprint 1,Sprinting Distance TIP 1,player_birthdate,match_name,Count HSR 1,Count HSR TIP,Count High Deceleration TIP,Count Medium Deceleration,Sprinting Distance 1,Count High Deceleration 1,Minutes OTIP 2,Count HSR TIP 2,Count High Deceleration OTIP,Sprinting Distance OTIP 1,Sprinting Distance 2,Count Medium Acceleration 1,Distance TIP 1,HSR Distance TIP,Running Distance TIP 1,Minutes 2,Count Medium Deceleration TIP 1,match_id,Distance OTIP 1,Count Medium Acceleration 2,Count Medium Deceleration 1,Count Medium Acceleration TIP,Count High Acceleration OTIP,Sprinting Distance TIP 2,Count HSR OTIP,Count High Acceleration OTIP 2,HSR Distance OTIP 2,Count Medium Acceleration OTIP 1,competition_edition_id,Count Sprint,Count Medium Acceleration,Sprinting Distance OTIP,Count Medium Acceleration TIP 1,HSR Distance OTIP,Count High Acceleration TIP 1,Sprinting Distance OTIP 2,Count Medium Deceleration OTIP,Minutes OTIP,Count Medium Deceleration 2,Running Distance,Running Distance 2,HSR Distance 2,Count High Acceleration OTIP 1,Count Sprint TIP,Running Distance OTIP 1,team_name,Count Medium Deceleration OTIP 2,season_name,group,Distance OTIP 2,player_name,Distance TIP 2,Count Medium Deceleration TIP,Sprinting Distance,HSR Distance TIP 2,Count Medium Deceleration TIP 2,Running Distance OTIP,quality_check,Count Medium Deceleration OTIP 1,Count Sprint OTIP,Minutes OTIP 1,Count Sprint TIP 1,short_name,team,Count High Acceleration 1,Count HSR OTIP 1,Count HSR,Minutes TIP 1,Running Distance TIP,Count High Deceleration OTIP 2,player_id,Running Distance 1,Count Medium Acceleration OTIP 2,Count High Acceleration,PSV-99,Count High Acceleration 2,HSR Distance TIP 1,Count High Acceleration TIP,Count Sprint OTIP 1,Count High Acceleration TIP 2,Count Medium Acceleration TIP 2,Minutes TIP,position,HSR Distance 1,Distance 2,Minutes 1,Count Sprint 2,Minutes TIP 2,Count HSR OTIP 2,Count High Deceleration TIP 1,date,Count High Deceleration,Count High Deceleration TIP 2,Running Distance TIP 2,competition_id,Count Sprint TIP 2,HSR Distance,Count HSR 2,Count HSR TIP 1,season_id,Distance OTIP,Count High Deceleration OTIP 1,Distance TIP,Count High Deceleration 2,Running Distance OTIP 2,Sprinting Distance TIP,HSR Distance OTIP 1,team_id,name,season_id_right,season_start_year,season_end_year,season_name_right,competition_id_right,competition_area,competition_name,competition_gender,competition_age_group
f64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,str,i64,str,i64,i64,i64,i64,i64,i64,bool,i64,i64,f64,i64,str,str,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,f64,str,i64,i64,f64,i64,f64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,str,i64,str,str,str,str
96.63,1,47,13026,6810,3,11,"""1995-02-17""","""KV Mechelen v Royal Antwerp FC""",33,19,3,89,92,8,17.3,6,4,27,56,52,1874,260,369,49.48,8,44999,2983,48,42,22,0,19,34,0,218,24,249,5,100,45,12,443,0,18,34,35.57,47,2415,1061,358,0,2,842,"""Royal Antwerp FC""",17,"""2021/2022""","""Midfield""",2744,"""Pieter Gerkens""",1558,17,148,121,9,1504,true,17,2,18.27,1,"""P. Gerkens""","""Royal Antwerp FC""",3,17,59,13.13,661,1,8701,1354,23,4,25.7,1,139,0,1,0,10,24.21,"""AM""",387,6216,47.15,2,11.08,17,2,"""2021-07-25""",11,1,292,16,1,745,26,13,8,5727,3,3432,3,662,30,225,462,"""BEL - Pro League - 2021/2022""",8,2021,2022,"""2021/2022""",16,"""BEL""","""Pro League""","""male""","""adult"""
96.19,0,32,9969,4842,3,17,"""1989-07-25""","""Oud-Heverlee Leuven v SV Zulte…",20,20,2,77,48,5,15.07,11,1,24,154,36,1660,265,267,50.07,9,45002,1795,41,31,25,1,119,17,1,63,15,249,8,77,24,11,167,1,0,25,29.59,46,1101,612,275,0,4,178,"""Oud-Heverlee Leuven""",14,"""2021/2022""","""Midfield""",1739,"""Xavier Mercier""",

In [27]:
# get the statsbomb levels
sb_cl = pl.read_csv('statsbomb-competition-levels.csv')

In [28]:
sb_cl.head()

competition_id,competition_name,competition_region_name,competition_level,competition_division_level
i64,str,str,i64,str
81,"""Liga Profesional""","""Argentina""",2,"""1"""
93,"""A-League""","""Australia""",3,"""1"""
47,"""Bundesliga""","""Austria""",2,"""1"""
1035,"""First Division B""","""Belgium""",3,"""2"""
46,"""Jupiler Pro League""","""Belgium""",2,"""1"""


In [29]:
# rename competition name for easy merge
sb_cl = sb_cl.rename({"competition_name":"sb_competition_name"})

In [30]:
sb_cl = sb_cl.drop(["competition_id", "competition_level"])

In [31]:
sb_cl.head()

sb_competition_name,competition_region_name,competition_division_level
str,str,str
"""Liga Profesional""","""Argentina""","""1"""
"""A-League""","""Australia""","""1"""
"""Bundesliga""","""Austria""","""1"""
"""First Division B""","""Belgium""","""2"""
"""Jupiler Pro League""","""Belgium""","""1"""


In [32]:
sb_sc = pl.read_csv('mapping-competition-seasons.csv')

In [33]:
sb_sc.head()

sb_competition_id,sb_competition_name,sb_season_id,sb_season_name,sc_competition_id,sc_competition_name,sc_season_id,sc_season_name,sc_competition_season_id,sc_competition_season_name
i64,str,i64,str,i64,str,i64,str,i64,str
2,"""Premier League""",317,"""2024/2025""",1,"""Premier League""",95,"""2024/2025""",895,"""ENG - Premier League - 2024/20…"
2,"""Premier League""",281,"""2023/2024""",1,"""Premier League""",28,"""2023/2024""",543,"""ENG - Premier League - 2023/20…"
2,"""Premier League""",235,"""2022/2023""",1,"""Premier League""",21,"""2022/2023""",387,"""ENG - Premier League - 2022/20…"
2,"""Premier League""",108,"""2021/2022""",1,"""Premier League""",8,"""2021/2022""",287,"""ENG - Premier League - 2021/20…"
2,"""Premier League""",90,"""2020/2021""",1,"""Premier League""",7,"""2020/2021""",147,"""ENG - Premier League - 2020/20…"


In [34]:
joined_sb_sc = sb_cl.join(sb_sc, on="sb_competition_name")

In [35]:
joined_sb_sc.head()

sb_competition_name,competition_region_name,competition_division_level,sb_competition_id,sb_season_id,sb_season_name,sc_competition_id,sc_competition_name,sc_season_id,sc_season_name,sc_competition_season_id,sc_competition_season_name
str,str,str,i64,i64,str,i64,str,i64,str,i64,str
"""Premier League""","""England""","""1""",2,317,"""2024/2025""",1,"""Premier League""",95,"""2024/2025""",895,"""ENG - Premier League - 2024/20…"
"""Premier League""","""Scotland""","""1""",2,317,"""2024/2025""",1,"""Premier League""",95,"""2024/2025""",895,"""ENG - Premier League - 2024/20…"
"""Premier League""","""Ukraine""","""1""",2,317,"""2024/2025""",1,"""Premier League""",95,"""2024/2025""",895,"""ENG - Premier League - 2024/20…"
"""Premier League""","""England""","""1""",2,281,"""2023/2024""",1,"""Premier League""",28,"""2023/2024""",543,"""ENG - Premier League - 2023/20…"
"""Premier League""","""Scotland""","""1""",2,281,"""2023/2024""",1,"""Premier League""",28,"""2023/2024""",543,"""ENG - Premier League - 2023/20…"


In [36]:
# extract columns containing skillcorner competition country and national division information, sc_competition_id
extracted_sb_sc = joined_sb_sc.select(["sc_competition_season_id", "competition_region_name", "competition_division_level"])

In [37]:
extracted_sb_sc = extracted_sb_sc.rename({"sc_competition_season_id":"competition_edition_id"})

In [38]:
extracted_sb_sc = extracted_sb_sc.with_columns(
    pl.concat_str(
        [
            pl.col("competition_region_name"),
            pl.col("competition_division_level"),
        ],
        separator = "_"
    ).alias("competition_region_division")
).drop(["competition_region_name", "competition_division_level"])

In [39]:
extracted_sb_sc.head()

competition_edition_id,competition_region_division
i64,str
895,"""England_1"""
895,"""Scotland_1"""
895,"""Ukraine_1"""
543,"""England_1"""
543,"""Scotland_1"""


In [40]:
final_df = work_df.join(extracted_sb_sc, on = "competition_edition_id")

In [41]:
final_df.shape

(508602, 128)

In [42]:
final_df.head()

Minutes,Count Sprint OTIP 2,Count Medium Acceleration OTIP,Distance,Distance 1,Count Sprint 1,Sprinting Distance TIP 1,player_birthdate,match_name,Count HSR 1,Count HSR TIP,Count High Deceleration TIP,Count Medium Deceleration,Sprinting Distance 1,Count High Deceleration 1,Minutes OTIP 2,Count HSR TIP 2,Count High Deceleration OTIP,Sprinting Distance OTIP 1,Sprinting Distance 2,Count Medium Acceleration 1,Distance TIP 1,HSR Distance TIP,Running Distance TIP 1,Minutes 2,Count Medium Deceleration TIP 1,match_id,Distance OTIP 1,Count Medium Acceleration 2,Count Medium Deceleration 1,Count Medium Acceleration TIP,Count High Acceleration OTIP,Sprinting Distance TIP 2,Count HSR OTIP,Count High Acceleration OTIP 2,HSR Distance OTIP 2,Count Medium Acceleration OTIP 1,competition_edition_id,Count Sprint,Count Medium Acceleration,Sprinting Distance OTIP,Count Medium Acceleration TIP 1,HSR Distance OTIP,Count High Acceleration TIP 1,Sprinting Distance OTIP 2,Count Medium Deceleration OTIP,Minutes OTIP,Count Medium Deceleration 2,Running Distance,Running Distance 2,HSR Distance 2,Count High Acceleration OTIP 1,Count Sprint TIP,Running Distance OTIP 1,team_name,Count Medium Deceleration OTIP 2,season_name,group,Distance OTIP 2,player_name,Distance TIP 2,Count Medium Deceleration TIP,Sprinting Distance,HSR Distance TIP 2,Count Medium Deceleration TIP 2,Running Distance OTIP,quality_check,Count Medium Deceleration OTIP 1,Count Sprint OTIP,Minutes OTIP 1,Count Sprint TIP 1,short_name,team,Count High Acceleration 1,Count HSR OTIP 1,Count HSR,Minutes TIP 1,Running Distance TIP,Count High Deceleration OTIP 2,player_id,Running Distance 1,Count Medium Acceleration OTIP 2,Count High Acceleration,PSV-99,Count High Acceleration 2,HSR Distance TIP 1,Count High Acceleration TIP,Count Sprint OTIP 1,Count High Acceleration TIP 2,Count Medium Acceleration TIP 2,Minutes TIP,position,HSR Distance 1,Distance 2,Minutes 1,Count Sprint 2,Minutes TIP 2,Count HSR OTIP 2,Count High Deceleration TIP 1,date,Count High Deceleration,Count High Deceleration TIP 2,Running Distance TIP 2,competition_id,Count Sprint TIP 2,HSR Distance,Count HSR 2,Count HSR TIP 1,season_id,Distance OTIP,Count High Deceleration OTIP 1,Distance TIP,Count High Deceleration 2,Running Distance OTIP 2,Sprinting Distance TIP,HSR Distance OTIP 1,team_id,name,season_id_right,season_start_year,season_end_year,season_name_right,competition_id_right,competition_area,competition_name,competition_gender,competition_age_group,competition_region_division
f64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,str,i64,str,i64,i64,i64,i64,i64,i64,bool,i64,i64,f64,i64,str,str,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,f64,str,i64,i64,f64,i64,f64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,str,i64,str,str,str,str,str
96.63,1,47,13026,6810,3,11,"""1995-02-17""","""KV Mechelen v Royal Antwerp FC""",33,19,3,89,92,8,17.3,6,4,27,56,52,1874,260,369,49.48,8,44999,2983,48,42,22,0,19,34,0,218,24,249,5,100,45,12,443,0,18,34,35.57,47,2415,1061,358,0,2,842,"""Royal Antwerp FC""",17,"""2021/2022""","""Midfield""",2744,"""Pieter Gerkens""",1558,17,148,121,9,1504,true,17,2,18.27,1,"""P. Gerkens""","""Royal Antwerp FC""",3,17,59,13.13,661,1,8701,1354,23,4,25.7,1,139,0,1,0,10,24.21,"""AM""",387,6216,47.15,2,11.08,17,2,"""2021-07-25""",11,1,292,16,1,745,26,13,8,5727,3,3432,3,662,30,225,462,"""BEL - Pro League - 2021/2022""",8,2021,2022,"""2021/2022""",16,"""BEL""","""Pro League""","""male""","""adult""","""Belgium_1"""
96.19,0,32,9969,4842,3,17,"""1989-07-25""","""Oud-Heverlee Leuven v SV Zulte…",20,20,2,77,48,5,15.07,11,1,24,154,36,1660,265,267,50.07,9,45002,1795,41,31,25,1,119,17,1,63,15,249,8,77,24,11,167,1,0,25,29.59,46,1101,612,275,0,4,178,"""Oud-Heverlee Leuven""",14,"""2021/2

In [43]:
final_df = final_df.select(sorted(final_df.columns))

In [44]:
# path to csv to write the dataframe to
csv_path = './data_summary.csv'

In [45]:
# writing  the dataframe to a csv
final_df.write_csv(csv_path)

In [ ]:
# extra columns taken to the next step
# competition_region_division
